In [53]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

# one hot encoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()

from pygam import LinearGAM, s, f, te

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

In [54]:
def same_direction(df, day_type):
    df_same = df[((df['WA_sentiment']<0) & (df[day_type]<0)) | ((df['WA_sentiment']>0) & (df[day_type]>0))]
    return df_same

# MD

In [55]:
df = pd.read_csv('./regression_df_input/MD_model_df.csv')
df_new = df.copy()
df_new['date'] = df_new['date'].astype(str)
df_new.head(5)

,company_name,Group,ticker,file_name,date,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.002680,0.572789,0.032527,0.000657,0.000499,0.044301,0.227743,0.008427,0.018461,0.037488,0.005803,0.048624,0.000392,1655.0,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.108180,0.505089,0.055883,0.030197,0.024078,0.056217,0.075847,0.025543,0.004550,0.042983,0.000752,0.070681,0.000765,1353.0,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.015945,0.415151,0.123024,0.001712,0.001302,0.028433,0.137670,0.042511,0.190022,0.001600,0.001559,0.041072,-0.000433,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.015842,0.691033,0.000680,0.000972,0.009756,0.040082,0.118486,0.017031,0.002626,0.076872,0.007984,0.018637,0.000336,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.009562,0.627906,0.088888,0.012241,0.011544,0.041281,0.123975,0.001258,0.029679,0.052195,0.000891,0.000580,0.001669,1144.0,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860


In [61]:
day_list = df_new.iloc[:,19:66].columns.to_list()
print(day_list)

['D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [59]:
# for day in day_list:
#     print(f"------------- {day}")
#     df_run = same_direction(df_new, day)
#     df_run['group'] = labelencoder.fit_transform(df_run['Group'])

#     X = df_run[['WA_sentiment','group']].values.tolist()
#     Y = df_run[day]

#     gam = LinearGAM(s(0)+f(1)).fit(X, Y)
#     print(gam.summary())
#     print("                      ")

In [67]:
X_col = ['WA_sentiment']
df_dummies = pd.get_dummies(df_new,prefix=['Group'], columns = ['Group'], drop_first=False)
dummies_category = df_dummies.columns[-10:]
print(len(dummies_category))
# print(df_new['Group'].nunique())
X_col.extend(dummies_category)
print('X_col:',X_col)

day_list = df_new.columns[17:].tolist()
print('day_list:',day_list)

10
X_col: ['WA_sentiment', 'Group_European (Re)Insurers', 'Group_Global Commercial', 'Group_Japanese & Pacific', 'Group_LN Equity', 'Group_Life Groups & Retail Life', 'Group_London Market', 'Group_Motor/Personal', 'Group_Other insurers', 'Group_Run-off', 'Group_US - Specialty/P&C/Reinsurance']
day_list: ['WA_sentiment', 'Close', 'D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [73]:
formula = 's(WA_sentiment) '
for cat in dummies_category:
    new_var = f'+ s({cat}) + te(WA_sentiment,{cat})'
    formula += new_var
print(formula)

for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_dummies, day)
    # df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[X_col].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0, n_splines=300) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)
                    + s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)
                    + s(9) + te(0,9)+ s(10) + te(0,10)).fit(X, Y)
    print(gam.summary())
    print("                      ")

s(WA_sentiment) + s(Group_European (Re)Insurers) + te(WA_sentiment,Group_European (Re)Insurers)+ s(Group_Global Commercial) + te(WA_sentiment,Group_Global Commercial)+ s(Group_Japanese & Pacific) + te(WA_sentiment,Group_Japanese & Pacific)+ s(Group_LN Equity) + te(WA_sentiment,Group_LN Equity)+ s(Group_Life Groups & Retail Life) + te(WA_sentiment,Group_Life Groups & Retail Life)+ s(Group_London Market) + te(WA_sentiment,Group_London Market)+ s(Group_Motor/Personal) + te(WA_sentiment,Group_Motor/Personal)+ s(Group_Other insurers) + te(WA_sentiment,Group_Other insurers)+ s(Group_Run-off) + te(WA_sentiment,Group_Run-off)+ s(Group_US - Specialty/P&C/Reinsurance) + te(WA_sentiment,Group_US - Specialty/P&C/Reinsurance)
------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDi

In [69]:
# formula = 's(0) '
# for i in range(len(dummies_category)):
#     new_var = f'+ s({i+1}) + te(0,{i+1})'
#     formula += new_var
# formula

# QA

In [74]:
df = pd.read_csv('./regression_df_input/QA_model_df.csv')
pd.set_option('display.max_columns', None)
#df_new = df[df.columns[:6]]
df_new = df.copy()
df_new['date'] = df_new['date'].astype(str)
df_new.head(5)

,company_name,Group,ticker,file_name,date,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.054848,0.158767,0.000649,0.121925,0.072709,0.158536,0.081791,0.001102,0.000962,0.019518,0.077290,0.251904,0.000826,1655.0,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.002386,0.158865,0.018418,0.034839,0.128776,0.252142,0.182165,0.067381,0.062381,0.027415,0.064643,0.000588,0.000971,1353.0,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.027649,0.058448,0.001426,0.004682,0.013177,0.135053,0.157797,0.037805,0.446674,0.068490,0.047127,0.001671,0.001851,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.000843,0.213648,0.004275,0.046098,0.087957,0.012637,0.069095,0.175282,0.050077,0.005097,0.072873,0.262119,0.001353,1144.0,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
4,Admiral Group,Motor/Personal,ADM.L,20130306_Admiral_Group_PLC-_Earnings_Call_2013...,2013-03-06,0.022662,0.750460,0.013020,0.016422,0.020791,0.034012,0.023247,0.024604,0.021481,0.035123,0.022911,0.015267,-0.046780,1334.0,0.052881,0.016492,0.045455,0.014243,0.069767,0.018741,0.066347,0.006747,0.033308,-0.001499,0.044636,-0.008996


In [75]:
day_list = df_new.iloc[:,19:66].columns.to_list()
print(day_list)

['D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [76]:
# # day = 'D+1'
# # df_run = same_direction(df_new, day)
# # # plot the df_run["WA_sentiment"] and df_run["D-1"]
# # plt.plot(df_run["WA_sentiment"], df_run["D+1"], '.')
# # plt.show()

# # # # run the OLS regression
# # # from sklearn.linear_model import LinearRegression
# # # import statsmodels.api as sm
# # # X = df_run[['WA_sentiment']].values.tolist()
# # # X = sm.add_constant(X)
# # # Y = df_run['D-1']


# # # model = sm.OLS(Y, X).fit()
# # # print(model.summary())

# # import plotly.express as px
# # fig = px.scatter(
# #     df_run, x='WA_sentiment', y='D+1', opacity=0.65,
# #     trendline='ols', trendline_color_override='darkblue',
# #     # trendline="rolling", trendline_options=dict(function="median", window=5)
# # )
# # fig.show()

# plt.figure()
# # fig, axs = plt.subplots(1,1)
# titles = ['sentiment']
# i = 0
# XX = gam.generate_X_grid(term=i)
# plt.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
# plt.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
# # plt.set_title(titles[i])

In [79]:
# day = "D-1"

# df_run = same_direction(df_new, day)
# df_run['group'] = labelencoder.fit_transform(df_run['Group'])

# X = df_run[['WA_sentiment']].values.tolist()
# Y = df_run[day]

# gam = LinearGAM(s(0, n_splines=500)).fit(X, Y)
# gam.summary()

In [80]:
# for day in day_list:
#     print(f"------------- {day}")
#     df_run = same_direction(df_new, day)
#     df_run['group'] = labelencoder.fit_transform(df_run['Group'])

#     X = df_run[['WA_sentiment','group']].values.tolist()
#     Y = df_run[day]

#     gam = LinearGAM(s(0)+f(1)).fit(X, Y)
#     print(gam.summary())
#     print("                      ")

In [81]:
X_col = ['WA_sentiment']
df_dummies = pd.get_dummies(df_new,prefix=['Group'], columns = ['Group'], drop_first=False)
dummies_category = df_dummies.columns[-10:]
X_col.extend(dummies_category)
print(X_col)
print(day_list)

['WA_sentiment', 'Group_European (Re)Insurers', 'Group_Global Commercial', 'Group_Japanese & Pacific', 'Group_LN Equity', 'Group_Life Groups & Retail Life', 'Group_London Market', 'Group_Motor/Personal', 'Group_Other insurers', 'Group_Run-off', 'Group_US - Specialty/P&C/Reinsurance']
['D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [82]:
formula = 's(WA_sentiment) '
for cat in dummies_category:
    new_var = f'+ s({cat}) + te(WA_sentiment,{cat})'
    formula += new_var
print(formula)

for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_dummies, day)
    # df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[X_col].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0, n_splines=300) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)
                    + s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)
                    + s(9) + te(0,9)+ s(10) + te(0,10)).fit(X, Y)
    print(gam.summary())
    print("                      ")

s(WA_sentiment) + s(Group_European (Re)Insurers) + te(WA_sentiment,Group_European (Re)Insurers)+ s(Group_Global Commercial) + te(WA_sentiment,Group_Global Commercial)+ s(Group_Japanese & Pacific) + te(WA_sentiment,Group_Japanese & Pacific)+ s(Group_LN Equity) + te(WA_sentiment,Group_LN Equity)+ s(Group_Life Groups & Retail Life) + te(WA_sentiment,Group_Life Groups & Retail Life)+ s(Group_London Market) + te(WA_sentiment,Group_London Market)+ s(Group_Motor/Personal) + te(WA_sentiment,Group_Motor/Personal)+ s(Group_Other insurers) + te(WA_sentiment,Group_Other insurers)+ s(Group_Run-off) + te(WA_sentiment,Group_Run-off)+ s(Group_US - Specialty/P&C/Reinsurance) + te(WA_sentiment,Group_US - Specialty/P&C/Reinsurance)
------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDi

In [ ]:
day = "D-1"
df_run = same_direction(df_dummies, day)
# df_run['group'] = labelencoder.fit_transform(df_run['Group'])
X = df_run[X_col].values.tolist()
Y = df_run[day]

gam = LinearGAM(s(0, n_splines=300) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)
                    + s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)
                    + s(9) + te(0,9)+ s(10) + te(0,10)).fit(X, Y)
print(gam.summary())
## plotting
plt.figure()
# fig, axs = plt.subplots(1,2)

titles = ['sentiment']
i = 0
XX = gam.generate_X_grid(term=i)
plt.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
plt.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
# ax.set_title(titles[i])

In [ ]:
# conclusion 
# TP use D+1
# MD use D-1